In [1]:
%pip install kaggle --upgrade split-folders

  Using cached kaggle-1.5.12.tar.gz (58 kB)
  Using cached split_folders-0.5.1-py3-none-any.whl (8.4 kB)
  Using cached python_slugify-6.1.1-py2.py3-none-any.whl (9.1 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73053 sha256=9d58437a5a22284e972962de4c4c67696b64d94534d8a82b839fcd80b21a6110
  Stored in directory: /home/azureuser/.cache/pip/wheels/77/47/e4/44a4ba1b7dfd53faaa35f59f1175e123b213ff401a8a56876b
Successfully built kaggle
Note: you may need to restart the kernel to use updated packages.


In [2]:
from azureml.core import Workspace, Dataset, Experiment

workspace = Workspace.from_config()

In [10]:
import json
# Export Kaggle configuration variables
%env KAGGLE_USERNAME=szelor
%env KAGGLE_KEY=21cd02df9ad71ca909694a3733b5b22d

env: KAGGLE_USERNAME=szelor
env: KAGGLE_KEY=21cd02df9ad71ca909694a3733b5b22d


In [11]:
!rm /tmp/chest-xray-pneumonia.zip
!rm -r /tmp/chest_xray
!rm -r /tmp/chest_xray_tvt

# Download the Pneumonia dataset. ISSUE: Requires Python 3.6 AzureML kernel which is not available in newer Compute Instances
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia -p /tmp

!unzip -q /tmp/chest-xray-pneumonia.zip -d /tmp

rm: cannot remove '/tmp/chest-xray-pneumonia.zip': No such file or directory
rm: cannot remove '/tmp/chest_xray': No such file or directory
rm: cannot remove '/tmp/chest_xray_tvt': No such file or directory
100%|██████████████████████████████████████▉| 2.29G/2.29G [00:29<00:00, 108MB/s]
100%|██████████████████████████████████████| 2.29G/2.29G [00:29<00:00, 84.4MB/s]


In [12]:
import splitfolders

download_root = '/tmp/chest_xray/train' 
train_val_test_root = '/tmp/chest_xray_tvt/'

train_val_test_split = (0.8, 0.1, 0.1)
random_seed = 33

splitfolders.ratio(download_root, train_val_test_root, random_seed, ratio=train_val_test_split)

Copying files: 5216 files [00:01, 2657.85 files/s]


In [ ]:
from azureml.core import Workspace, Datastore, Dataset
from azureml.data.datapath import DataPath

# Upload data to AzureML Datastore
ds = workspace.get_default_datastore()
ds = Dataset.File.upload_directory(src_dir=train_val_test_root,
            target=DataPath(ds, 'chest-xray'),
            show_progress=False, overwrite=False)

# Register file dataset with AzureML
ds = ds.register(workspace=workspace, name="pneumonia", description="Pneumonia train / val / test folders with 2 classes", create_new_version=True)

print(f'Dataset {ds.name} registered.')